# Starbucks Capstone Challenge: Data preparation

## Contents
1. Remove missing demographic data. 
2. Convert numerical data to categorical data.
3. Compute target variable (responsiveness). 
4. Combine prepared feature and target data.
5. Output prepared dataset.

In [1]:
# Importing packages 
import pandas as pd
import pickle

In [2]:
# Read in json files
portfolio = pd.read_json('../data/raw/portfolio.json', orient='records', lines=True)
profile = pd.read_json('../data/raw/profile.json', orient='records', lines=True)
transcript = pd.read_json('../data/raw/transcript.json', orient='records', lines=True)

### 1. Remove missing demographic data

In [3]:
# Rename column 'id' to 'person' to remain consistent with transcript
profile = profile.rename(columns={'id':'person'})

# Remove people was ages > 99
profile = profile[profile['age'] <= 99]

# Remove people with income > 1000000
profile = profile[profile['income'] < 1000000.0]

### 2. Feature engineering: convert numerical data to categorical data

In [4]:
def cat_age(age):
    '''
    Categorises age into age group.
    '''
    if age < 25:
        return '< 25 years'
    
    if age < 35:
        return '25 - 35 years'
    
    if age < 45:
        return '36 - 45 years'
    
    if age < 66: 
        return '46 - 66 years'
    
    else: 
        return '67+ years'

In [5]:
# Apply cat_age function to age column
profile['age_group'] = profile['age'].apply(lambda x:cat_age(x))

In [6]:
def cat_income(income):
    '''
    Categorises income into income group.
    '''
    if income < 50001: 
        return 'low income'
    if income < 70001:
        return 'med - low income'
    if income < 90001:
        return 'high - med income'
    else:
        return 'high income'

In [7]:
# Apply cat_age function to age column
profile['income_group'] = profile['income'].apply(lambda x:cat_income(x))

In [8]:
# Select demographic data columns for training model
profile_subset = profile[['person','income_group','age_group', 'gender']]

# Reset index so that person would be a column. Profile_subset will be merged on person column 
profile_subset = profile_subset.reset_index()

# Drop additionally index column produced from reset index step above
profile_subset.drop('index', axis=1, inplace=True)

### 3. Computing user responsiveness

In [9]:
# Get offer ids from 'value' column, convert to float, and store in new column
offer_ids = dict()
indx = list(transcript[transcript['event']!='transaction'].index)

for ind in indx: 
    offer_id = list(transcript.iloc[ind]['value'].values())[0]
    offer_ids.update({ind:offer_id})
    
# Make dataframe from dictionary of index, offer_id strings     
offer_id_df = pd.DataFrame.from_dict(offer_ids, orient='index', columns=['offer_ids'])

In [10]:
# Concat transcript_mod and offer_id_df dataframes
transcript_mod = pd.concat([transcript, offer_id_df], axis=1, ignore_index=False)

# rename column 'id' as offer_ids to remain consistent with transcript df
portfolio = portfolio.rename(columns={'id':'offer_ids'})

In [11]:
# Merge transcript and portfolio dataframes
transcript_portfolio = transcript_mod.merge(portfolio[['offer_ids', 'offer_type']], on='offer_ids', how='left')

offers = ['bogo', 'discount']

# Filter transcript_portfolio to get transcripts corresponding to BOGO and discount offers 
offer = transcript_portfolio[transcript_portfolio['offer_type'].isin(offers)]

# Remove columns that will not used for the classifier
offer.drop(['value','time', 'offer_ids'], axis=1, inplace=True)

/Users/SamaraSadeek/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
# Calculate number of offers completed and viewed per person 
offer_per_person = offer.groupby(['person','offer_type'])['event'].value_counts().unstack()

# Fill missing offer completed data with 0
offer_per_person['offer completed'].fillna(0, inplace=True)

In [13]:
# Calculate proportion of offers viewed that were compeleted for each user 
offer_per_person['completed_per_view'] = offer_per_person['offer completed']/ offer_per_person['offer viewed']

# Replace missing proportion with 0
offer_per_person['completed_per_view'].fillna(0, inplace=True)

In [14]:
def set_response(cpv):
    '''
    Return a binary label for whether or not a user is responsive. 
    A responsive user is someone who has completed more than 50 % of offers
    received. 
    '''
    if cpv <= 0.5: 
        return 0 
    else: 
        return 1

In [15]:
# Apply set_response function to completed_per_view column
offer_per_person['responds'] = offer_per_person['completed_per_view'].apply(lambda x:set_response(x))

In [16]:
# Keep only the BOGO and discount responsiveness column
offer_df = offer_per_person.unstack()

offer_df = offer_df['responds']

In [17]:
# Drop users with missing responsivness label
offer_df.dropna(inplace=True)

# Reset index so person is a column to enable merging demographic and offer data
offer_df = offer_df.reset_index()

### 4. Combining feature variables and labels 

In [18]:
# Merge prepared offer and demographic data
offer_per_person_demo = offer_df.merge(profile_subset, left_on='person',right_on='person', how='left').set_index('person')

### 5. Output prepared dataset

In [19]:
# Pickle prepared dataframe
offer_per_person_demo.to_pickle("../data/processed/prepared_data.pkl") 